# Soil Agent

In [ ]:
# Cell 1 (continued): Imports & env
import os
import json
from typing import TypedDict, Optional, Literal, Dict, Any
from datetime import datetime
from dotenv import load_dotenv
from together import Together
from langgraph.graph import StateGraph

load_dotenv()


True

In [ ]:
Together_api_key=os.getenv("TOGETHER_API_KEY")
client=Together()
MODEL_NAME="Qwen/Qwen2.5-72B-Instruct-Turbo"

In [ ]:
llama_model_name="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"


In [ ]:
from typing import Optional, Literal
from dataclasses import dataclass
    
@dataclass
class FarmerInput:
    crop_name: str                     # e.g. Wheat, Soybean
    crop_variety: Optional[str] = ""   # Lokwan, Pusa-322 etc
    location: str = ""                 # Village + District
    sowing_date: Optional[str] = None  # "YYYY-MM-DD"
    soil_type: Optional[Literal["light","medium","heavy"]] = None
    area: Optional[float] = None       # acres / hectare
    irrigation_type: Optional[Literal["rainfed","borewell","canal","drip"]] = "rainfed"

In [ ]:
SOIL_SYSTEM_PROMPT = """You are the SOIL AGENT in a multi-agent agricultural system.

INPUTS:
- Crop Name, Variety, Sowing Date, Location, Area
- Soil Report (optional)

YOUR TASK:
Provide baseline soil data for the given location. Keep it factual and numerical.

OUTPUT FORMAT:

1. SOIL TYPE: based on location
2. pH: [X.X] - [Acidic/Neutral/Alkaline]
3. NPK LEVELS (kg/ha):
   - N: [XXX] - [Low/Medium/High]
   - P: [XX] - [Low/Medium/High]  
   - K: [XXX] - [Low/Medium/High]
4. ORGANIC CARBON: [X.X%] - [Low/Medium/High]
5. CRITICAL MICRONUTRIENTS (ppm):
   - Zn: [X.X] - [Deficient/Adequate]
   - Fe: [X.X] - [Deficient/Adequate]
6. SOIL TEXTURE & WATER RETENTION:
   [Sandy/Loamy/Clayey] - WHC: [XX%] or [Poor/Moderate/Good]
7. SOIL HEALTH SCORE: [X.X/5]

RULES:
✅ If NO report → estimate based on typical soil characteristics of the location/district
   - Use regional soil databases (NBSS&LUP, state agriculture dept data)
   - Consider common soil types for that area
   - Apply average nutrient values for similar regions
✅ Always give numbers first, then category
✅ Mention if values are "Estimated" or "From Report"
❌ No fertilizer advice
❌ No crop recommendations
❌ No explanations - just data

This data feeds into downstream agents (fertilizer, irrigation, pest).
"""


In [ ]:
def run_soil_agent(
    location: str,
    soil_type: str = "",
    has_soil_report: bool = False,
    soil_report_text: str = ""
)-> str:
    """
    Calls the SOIL AGENT and returns raw six-line text with:
    pH, N, P, K, Organic Carbon, Soil Health Score
    """

    user_msg = f"""
    location: {location}
    soil_type: {soil_type}
    has_soil_report: {has_soil_report}
    soil_report_text: {soil_report_text}
    """

    resp = client.chat.completions.create(
        # model=MODEL_NAME,
        model=llama_model_name,
        messages=[
            {"role": "system", "content": SOIL_SYSTEM_PROMPT},
            {"role": "user", "content": user_msg},
        ],
        max_tokens=256,
        temperature=0.2,
    )

    text = resp.choices[0].message.content.strip()
    return text


In [ ]:
user_input=FarmerInput(location="dhar ",crop_name="wheat",sowing_date="11/11/2025")

In [ ]:
soil_data=run_soil_agent(user_input)
print("------------------------------------limbodi soil report using llama model ---------------------")
print(soil_data)

------------------------------------limbodi soil report using llama model ---------------------
Based on the location 'Limbodi, Madhya Pradesh', here's the estimated baseline soil data:

1. SOIL TYPE: Inceptisol (Estimated)
2. pH: 7.2 - Neutral (Estimated)
3. NPK LEVELS (kg/ha):
   - N: 120 - Medium (Estimated)
   - P: 15 - Medium (Estimated)
   - K: 180 - Medium (Estimated)
4. ORGANIC CARBON: 0.4% - Low (Estimated)
5. CRITICAL MICRONUTRIENTS (ppm):
   - Zn: 0.8 - Deficient (Estimated)
   - Fe: 4.5 - Adequate (Estimated)
6. SOIL TEXTURE & WATER RETENTION:
   Loamy - WHC: 30% or Moderate (Estimated)
7. SOIL HEALTH SCORE: 2.8/5 (Estimated)

Note: These values are estimated based on typical soil characteristics of the region and may not reflect the actual soil conditions at the specific location.


In [ ]:
soil_data=run_soil_agent(user_input)
print("------------------------------------limbodi soil report using qwen model ---------------------")
print(soil_data)

------------------------------------limbodi soil report using qwen model ---------------------
1. SOIL TYPE: Inceptisol (Estimated)
2. pH: 7.8 - Alkaline (Estimated)
3. NPK LEVELS (kg/ha):
   - N: 120 - Medium (Estimated)
   - P: 18 - Medium (Estimated)
   - K: 150 - Medium (Estimated)
4. ORGANIC CARBON: 0.6% - Medium (Estimated)
5. CRITICAL MICRONUTRIENTS (ppm):
   - Zn: 0.8 - Deficient (Estimated)
   - Fe: 4.5 - Adequate (Estimated)
6. SOIL TEXTURE & WATER RETENTION:
   Loamy - WHC: 25% or Moderate (Estimated)
7. SOIL HEALTH SCORE: 3.2/5 (Estimated)

Note: The values are estimated based on typical soil characteristics of the Limbodi region in Madhya Pradesh.


# Water Agent

<!-- Water Agent -->

In [ ]:
water_system_prompt = """
You are the WATER AGENT.

Task:
Given only a location, a crop name, and an optional water source, return a concise plain-text regional water-quality summary useful for farmers. Do NOT use soil data, do NOT create irrigation schedules, and do NOT output JSON.

Important behavior:
- Return only the water-related information that can be reasonably inferred from the given location and water source. 
- If a specific field cannot be inferred regionally, do NOT include that field in the output at all (do NOT write 'Unknown' or placeholders).
- Be conservative: when you must guess from regional trends, use lower confidence (<=0.6) and keep language tentative (e.g., "often", "commonly", "may").
- Keep output concise and practical for farmers. Maximum ~15 lines.

Preferred labels (print only those that have values):
Location: <location>
Crop: <crop>

Suitability for irrigation: <Good / Moderate / Poor>
Water availability: <High / Medium / Low>
Water source type: <groundwater / canal / river / rainfed>
Common local water issues: <short list, e.g., "Fluoride; Salinity">
General EC tendency: <Low / Moderate / High>
TDS (typical range): <e.g., 200–600 mg/L>
Hardness (Ca+Mg): <Soft / Moderate / Hard>




Confidence: <0.0–1.0>

Rules:
- Plain text only, no JSON.
- Only include labels/lines for which you can provide a meaningful, regionally-informed answer. Omit others completely.
- Do NOT mention soil, fertilizers, nutrients, or create irrigation schedules.
- If you are inferring a value (not from lab data), make it clear in one short word or phrase (e.g., "commonly", "often", "may") and use a lower confidence.
"""


In [ ]:
def water_agent(farmer: FarmerInput) -> str:
    """
    Water Agent that uses FarmerInput dataclass.
    """
    location = farmer.location
    crop = farmer.crop_name
    water_source = farmer.irrigation_type if farmer.irrigation_type else ""

    user_msg = f"""
    location: {location}
    crop: {crop}
    water_source: {water_source}

    Please respond only with the plain-text water-quality summary following the system prompt format.
    """

    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": water_system_prompt},
            {"role": "user", "content": user_msg},
        ],
        max_tokens=300,
        temperature=0.2,
    )

    water_text= resp.choices[0].message.content.strip()
    return water_text

In [ ]:
water_data=water_agent(user_input)
print("--------------------------------water agent res using qwen modle -----------------------------")
print(water_data)

--------------------------------water agent res using qwen modle -----------------------------
Location: Limbodi, Madhya Pradesh
Crop: Wheat

Suitability for irrigation: Moderate
Water availability: Medium
Water source type: rainfed
Common local water issues: Salinity; Fluoride
General EC tendency: Moderate
TDS (typical range): 300–800 mg/L
Hardness (Ca+Mg): Moderate

Confidence: 0.6


In [ ]:
water_data=water_agent(water_user_input)
print("--------------------------------water agent res using llama modle -----------------------------")
print(water_data)

--------------------------------water agent res using llama modle -----------------------------
Location: Limbodi, Madhya Pradesh
Crop: Wheat

Water source type: rainfed
Suitability for irrigation: Poor (often dependent on rainfall)
Water availability: Low (commonly variable)
Common local water issues: Fluoride
General EC tendency: Moderate
TDS (typical range): 300–500 mg/L
Hardness (Ca+Mg): Hard

Confidence: 0.55
